# 구글 리뷰 - 크롤링 수집 작업 과정

구글 리뷰 크롤링 수집 작업은 다수의 데이터 수집을 하면서 발생한 문제들을 단계적으로 해결하며 발전했습니다. 

### 1. 초기 버전: Selenium을 이용한 단순 스크롤 및 리뷰 수집.
- 문제점: 스크롤을 할 때마다 이전에 로드된 리뷰가 중복 수집되는 문제 발생.

### 2. v2: 선(先) 전체 스크롤, 후(後) 일괄 수집 방식으로 변경.
- 개선점: 중복 수집 문제 해결.
- 새로운 문제점: 수집 데이터량이 많아서 스크롤을 몇번 해야 지정한 날짜까지 데이터가 수집되는지를 확인 불가. 
   
### 3. v3: 목표 날짜 도달 시 스크롤을 중단하는 로직 추가.
- 개선점: 수집 시간 단축.
- 새로운 문제점: 수집 데이터량이 많을 경우, 일괄 수집 전 메모리 초과로 프로세스가 중단되는 안정성 문제 발견.

### 4. v4(크롤링 final 버전): 중간 저장 기능 추가.
- 개선점: 일정량의 스크롤마다 수집된 데이터를 파일에 중간 저장하여, 프로세스가 중단되더라도 재개가 가능하도록 안정성 확보.
- 새로운 문제점: v3보다 보다 많은 데이터 수집은 가능하지만, ui를 그리면서 작업이 진행되기 때문에 메모리 초과 문제가 완전히 해결되지 않음.
   
### <b>5. final 버전[colab 작업](방식 변경): google-play-scraper 라이브러리 도입</b>
- 근본적인 해결: Selenium 기반 UI 자동화의 근본적인 한계(속도, 안정성)를 인지하고, API 방식으로 데이터를 직접 요청하는 google-play-scraper로 방법론을 전면 교체.
- 최종 결과: 메모리 문제와 속도 문제를 근본적으로 해결하여, 안정적이고 빠른 대규모 리뷰 수집 시스템을 완성.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

import pandas as pd
from datetime import datetime

In [ ]:
# ==========================================================
# 1. 설정
# ==========================================================
# 웹 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument("lang=ko_KR")
browser = webdriver.Chrome(options=options)
wait = WebDriverWait(browser, 20) 

# 검색할 키워드 및 데이터를 저장할 파일 이름 설정
keyword = "카카오톡"
csv_filename = "kakao_reviews_final.csv"

try:
    # ==========================================================
    # 2. 기존 데이터 불러오기 (이어하기)
    # ==========================================================
    processed_reviews_body = set() 
    try:
        # 기존 저장한 CSV 파일을 읽어서 processed_reviews_body를 미리 채워 넣기.
        print(f"기존에 수집한 데이터를 불러옵니다: '{csv_filename}'")
        existing_df = pd.read_csv(csv_filename)
        processed_reviews_body.update(existing_df['본문'].tolist())
        print(f"총 {len(processed_reviews_body)}개의 기존 리뷰를 호출했습니다. 새 리뷰만 수집합니다.")
        is_first_save = False
    except FileNotFoundError:
        print("기존 데이터 파일이 없습니다. 처음부터 수집을 시작합니다.")
        is_first_save = True

    # ==========================================================
    # 3. 구글 플레이 스토어 접속 및 리뷰 팝업 열기
    # ==========================================================
    url = f"https://play.google.com/store/search?q={keyword}&c=apps&hl=ko"
    browser.get(url)
    time.sleep(2) # 페이지 로딩 대기
    
    # 상세 페이지로 이동
    app_url = browser.find_element(By.CLASS_NAME, 'Qfxief').get_attribute('href')
    browser.get(app_url)
    time.sleep(2) # 상세 페이지 로딩 대기
    
    # '리뷰 모두 보기' 버튼 클릭
    print("'리뷰 모두 보기' 버튼을 클릭합니다.")
    browser.find_elements(By.CLASS_NAME, 'VfPpkd-Bz112c-LgbsSe')[7].click()
    
    # '최신' 정렬 버튼 클릭
    print("'최신' 정렬 버튼을 클릭합니다.")
    sort_button = wait.until(EC.element_to_be_clickable((
        By.XPATH, "//div[@role='button' and @aria-label='최신']"
    )))
    browser.execute_script("arguments[0].click();", sort_button)
    time.sleep(3)

    # ==========================================================
    # 4. 스크롤하며 실시간 수집 및 저장
    # ==========================================================
    target_date_str = "2025년 10월 2일" 
    target_date = datetime.strptime(target_date_str, "%Y년 %m월 %d일")
    print(f"'{target_date_str}' 이전 리뷰가 나타날 때까지 스크롤하며 수집을 시작합니다.")

    scrollable_div = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'fysCi')))
    
    while True:
        reviews_on_screen = browser.find_elements(By.CLASS_NAME, 'RHo1pe')
        
        review_batch = []
        for review in reviews_on_screen:
            try:
                본문 = review.find_element(By.CLASS_NAME, 'h3YV2d').text
                if 본문 not in processed_reviews_body:
                    이름 = review.find_element(By.CLASS_NAME, 'X5PpBb').text
                    평점문자 = review.find_element(By.CLASS_NAME, 'iXRFPc').get_attribute("aria-label")
                    평점 = ''.join(filter(str.isdigit, 평점문자))[-1] 
                    날짜 = review.find_element(By.CLASS_NAME, 'bp9Aid').text
                    try:
                        평가문자 = review.find_element(By.CLASS_NAME, 'AJTPZc').text
                        평가 = ''.join(filter(str.isdigit, 평가문자))
                    except:
                        평가 = "0"
                    review_data = {"이름": 이름, "평점": 평점, "날짜": 날짜, "본문": 본문, "평가": 평가}
                    review_batch.append(review_data)
                    processed_reviews_body.add(본문)
            except Exception:
                continue
        
        if review_batch:
            print(f"새로운 리뷰 {len(review_batch)}개 발견. 파일에 저장합니다. [현재까지 총 수집량: {len(processed_reviews_body)}개]")
            temp_df = pd.DataFrame(review_batch)
            if is_first_save:
                temp_df.to_csv(csv_filename, encoding="utf-8-sig", index=False, mode='w')
                is_first_save = False
            else:
                temp_df.to_csv(csv_filename, encoding="utf-8-sig", index=False, mode='a', header=False)

        last_review_date_str = reviews_on_screen[-1].find_element(By.CLASS_NAME, 'bp9Aid').text
        last_review_date = datetime.strptime(last_review_date_str, "%Y년 %m월 %d일")
        if last_review_date <= target_date:
            print(f"목표 날짜({target_date_str})에 도달하여 수집을 종료합니다.")
            break
        
        last_height = browser.execute_script("return arguments[0].scrollHeight", scrollable_div)
        browser.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", scrollable_div)
        time.sleep(2)
        new_height = browser.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:
            print("페이지 끝에 도달하여 수집을 종료합니다.")
            break
    
    print(f"\n모든 작업이 완료되었습니다. 데이터가 '{csv_filename}' 파일에 저장되었습니다.")

except Exception as e:
    print(f"\n스크립트 실행 중 오류가 발생했습니다: {e}")

finally:
    print("브라우저를 종료합니다.")
    browser.quit()